In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

In [ ]:
def convert_format2D(arr, N):
    ret = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            ret[i,j] = arr[i + N * j]
    
    return ret

def read_snapshots2D(N):
    snapshots = []
    count = 0
    done = False
    while not done:
        try:
            phi1 = convert_format2D(np.fromfile(f"phi1-snapshot{count}", dtype=np.float32), N)
            phi2 = convert_format2D(np.fromfile(f"phi2-snapshot{count}", dtype=np.float32), N)
            # saxion = np.sqrt(phi1**2 + phi2**2)
            axion = np.arctan2(phi1, phi2)
            snapshots.append(axion)
            count += 1
        except:
            done = True
    
    return snapshots

def convert_format3D(arr, N):
    ret = np.zeros((N, N, N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                ret[i,j,k] = arr[(i * N + j) * N + k]

    return ret

def read_snapshots3D(N):
    snapshots = []
    count = 0
    done = False
    while not done:
        try:
            phi1 = convert_format3D(np.fromfile(f"phi1-snapshot{count}", dtype=np.float32), N)
            phi2 = convert_format3D(np.fromfile(f"phi2-snapshot{count}", dtype=np.float32), N)
            # saxion = np.sqrt(phi1**2 + phi2**2)
            axion = np.arctan2(phi1, phi2)
            snapshots.append(axion)
            count += 1
        except:
            done = True
    
    return snapshots

In [ ]:
# For nicer plots:

%matplotlib inline
font = {'size'   : 20, 'family':'STIXGeneral'}
axislabelfontsize='large'
matplotlib.rc('font', **font)
matplotlib.mathtext.rcParams['legend.fontsize']='small'
matplotlib.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['figure.figsize'] = [13, 12]

In [ ]:
# Show 2D snapshots:

snapshots = read_snapshots2D(256)

for i in range(len(snapshots)):
    plt.imshow(snapshots[i], cmap=matplotlib.cm.twilight, interpolation='none')
    plt.title(r"Axion field $a(x)/f_a$")
    # plt.xlabel(r"Comoving distance $\tilde{x}$")
    locx, _ = plt.xticks([])
    locy, _ = plt.yticks([])
    cbar = plt.colorbar(fraction=0.046, pad=0.04)
    cbar.set_ticks([np.pi-0.001, np.pi/2, 0, -np.pi/2, -np.pi+0.001])
    cbar.set_ticklabels([r"$\pi$", r"$\pi/2$","0",r"$-\pi/2$", r"$-\pi$"])
    plt.show()


In [ ]:
# Show 3D snapshots as 2D slices:

snapshots = read_snapshots3D(128)

for i in range(len(snapshots)):
    plt.imshow(snapshots[i][0], cmap=matplotlib.cm.twilight, interpolation='none')
    plt.title(r"Axion field $a(x)/f_a$")
    # plt.xlabel(r"Comoving distance $\tilde{x}$")
    locx, _ = plt.xticks([])
    locy, _ = plt.yticks([])
    cbar = plt.colorbar(fraction=0.046, pad=0.04)
    cbar.set_ticks([np.pi-0.001, np.pi/2, 0, -np.pi/2, -np.pi+0.001])
    cbar.set_ticklabels([r"$\pi$", r"$\pi/2$","0",r"$-\pi/2$", r"$-\pi$"])
    plt.show()


In [ ]:
# Read single snapshot:

p1 = convert_format3D(np.fromfile("phi1-snapshot3", dtype=np.float32), 256)
p2 = convert_format3D(np.fromfile("phi2-snapshot3", dtype=np.float32), 256)

ax = np.arctan2(p1,p2)
sx = np.sqrt(p1**2 + p2**2)

plt.imshow(ax[80], cmap=matplotlib.cm.twilight, interpolation='none')
plt.title(r"Axion field $a(x)/f_a$")
locx, _ = plt.xticks([])
locy, _ = plt.yticks([])
cbar = plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_ticks([np.pi-0.001, np.pi/2, 0, -np.pi/2, -np.pi+0.001])
cbar.set_ticklabels([r"$\pi$", r"$\pi/2$","0",r"$-\pi/2$", r"$-\pi$"])
plt.show()

In [ ]:
# Time-series observables:
plt.rcParams['figure.figsize'] = [6.5, 6]

ret = pd.read_csv("time-series.csv")

t = np.array(ret['time'].tolist())
xi = np.array(ret['xi'].tolist())
phi1_bar = np.array(ret['phi1_bar'].tolist())
phi2_bar = np.array(ret['phi2_bar'].tolist())
phidot1_bar = np.array(ret['phidot1_bar'].tolist())
phidot2_bar = np.array(ret['phidot2_bar'].tolist())
axion_bar = np.arctan2(phi1_bar, phi2_bar)
saxion_bar = np.sqrt(phi1_bar**2 + phi2_bar**2)

plt.plot(t, xi, "x")
plt.show()

plt.scatter(phi1_bar, phi2_bar, marker=".", c=t, cmap=matplotlib.cm.viridis)
plt.show()

plt.plot(t, phi1_bar, "x", label="phi1")
plt.plot(t, phi2_bar, "x", label="phi2")
plt.plot(t, phidot1_bar, "x", label="phidot1")
plt.plot(t, phidot2_bar, "x", label="phidot1")
plt.legend()
plt.show()

plt.scatter(phi1_bar, phidot1_bar, marker=".", c=t, cmap=matplotlib.cm.viridis, label="phi1")
plt.scatter(phi2_bar, phidot2_bar, marker="x", c=t, cmap=matplotlib.cm.viridis, label="phi2")
plt.legend()
plt.show()

In [ ]:
# Plot string snapshot:

from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook
plt.rcParams['figure.figsize'] = [8, 8]

ret = pd.read_csv("string-pos-snapshot4", header=None)
xs = ret[0].tolist()
ys = ret[1].tolist()
zs = ret[2].tolist()

ax = plt.axes(projection='3d')
ax.scatter3D(xs, ys, zs, c=zs, cmap=matplotlib.cm.viridis);
plt.show()